# Geometric Computer Vision 236861 Final Project
## Winter 2023-2024
## by: Eldad Matmon and Ehud Gordon

<img src="figures/360_1AM.gif" width="1280" align="center">

# Introduction
In July 2023, ["3D Gaussian Splatting for Real-Time Radiance Field Rendering"](https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/) by Inria suggested Gaussian Splatting (GS) as a high-quality real-time alternative to Novel View Synthesis (NVS). The method optimizes 3D Gaussians for scene representation. However, for views that are far from the input data, the base implementation of GS produces blurry images, due to extrapolation in regions the 3D Gaussians weren't optimized for.  
Our aim in this project is to improve Novel View Synthesis with GS for such views. We do so by generating realistic novel views using a diffusion model, thus providing supervision in areas with little coverage in training. 

# Gaussian Splatting

<div style="text-align: center">
    <img src="figures/colmap.png" style="width:100%; margin-bottom: 50px;" >
</div>

<div style="text-align: center">
    <img src="figures/GS_flow.png" style="width:100%; margin-bottom: 50px; height:400px;" >
</div>



<div style="text-align: center">
    <img src="figures/GS_gaussian.png" style="width:100%; margin-bottom: 50px;" >
</div>



<div style="text-align: center">
    <img src="figures/GS_Bear.png" style="width:100%; margin-bottom: 20px;" >
</div>

## Motivation - Degraded Rendering in Sparse-View
In the figure below (center) is the output of the vanilla implementation of a Gaussian Splatting model on a view far from the training data. As can be seen, Base GS produces blurry output for such views. However, the model is fully capable of high-quality images. On the right, we show the output of a base model on one of its training images. 

<img src="figures/gt_vs_coarse_vs_coarse12/comparison_0_2.png">

# Method
In order to enhance visual consistency from unseen viewpoints, we've implemented the following scheme. . First, we've used a visual hull with floater elimination to generate initial gaussians.
## Initialization (Visual Hull)
We've implemented a visual hull method to caputre initial conditions for the image. 
A visual hull is a geometric approximation of an object’s 3D shape, constructed by intersecting the viewing cones formed by multiple 2D silhouettes (masks) of the object taken from different viewpoints. The masks were obtained using SAM model from meta. Each silhouette/mask is treated as a projection of the object onto the corresponding image plane, and the regions in 3D space that can project to all these silhouettes are considered part of the visual hull. Further, the colors of the 3D points can be averaged from the images, allowing to initialize both position and color for the 3D Gaussians.
<div style="text-align: center">
    <img src="figures/visualHull.png" style="width:100%; margin-bottom: 20px; height:400px" >
</div>
## 
The training scheme is displayed in the figure below. In the first stage (top part), we've trained a GS-model following the implementation suggested in the original paper. Specifically, (differentiable) renderings are produced using GS from views in the training sets. The Gaussians parameters are optimized to minimize L1-loss and DSSIM-loss between those outputs and the training images. By the end of this stage, the GS-model produces high-quality output for near-training views, but blurry output for unseen views. To address this issue, we introduce a further stage (lower part): The final stage takes the output of the previous stage, and continues the Gaussians' optimization, however this time producing renderings from both training views, and novel views. As those novel views have no ground truth, we pass the renderings of GS into a diffusion-model, pretrained on cleaning Gaussian Noise.  We use a ControlNet model, as it allows us to steer the diffusion-model output so it'll produce an output of the same instance shown in the input. We then optimize the Gaussians using the loss between the diffusion-model output and the GS-model output.

<div style="text-align: center">
    <img src="figures/gaussianObjectDiagram.png" />
</div>

# Evaluation
## Test Setup
As training data, we took four images (red frustums in figure below), covering 360 degrees, of an object lying on the ground, from a normal height (1.5 meters). We call this "Top". As a test set, we took 8 views (blue frustums in figure below) of the object from the "Bottom" - around 0.5 meter from the ground. The setup is displayed below.

<div style="text-align: center">
    <img src="figures/test_train_cameras.gif" />
</div>


## Results
We now compare the output of a Base GS, trained on 10000 iterations, with the output of our model, trained on novel views for another 4000 iterations, using supervision from a diffusion-model. The results are displayed in the figure at the top of the report, and below:

<div style="text-align: center">
    <div style="position: relative; width: 100%; height: 20px; margin-bottom: 10px;">
        <span style="position: absolute; left: 50%; transform: translateX(-300px);">Base GS</span>
        <span style="position: absolute; left: 50%; transform: translateX(200px);">Ours</span>
    </div>
    <img src="figures/gt_vs_go/comparison_0.png" />
    <img src="figures/gt_vs_go/comparison_2.png" />
    <img src="figures/gt_vs_go/comparison_3.png" />

</div>

As can be seen, the output is inconsistent. Whereas sometimes it produces sharp, visually-appealing results, in other cases it outputs highly blurred images. When inspecting the diffusion-model output, we've noticed that while on some views it outputs decent results, as shown in the #Method section diagram, on other  views the diffusion-model hallucinates, and produces double objects, as shown below:
<div style="text-align: center">
    <img src="figures/controlnet_hallucinations/model_input.png" style="display: block; margin: auto;"/>
    <img src="figures/controlnet_hallucinations/model_output.png" style="display: block; margin: auto;"/>
</div>

We discuss this issue in the Conclusion section.

As for metrics, the PSNR of the base GS on the test dataset is around 14.5, and ours is 15.1 

# Experiments
We've tested two alternatives to the diffusion model. 
### (1) Unsharp Masking
We've implemented  Unsharp Masking using a two-stage approach. Initially we've employed a Gaussian blur to smoothen the image, and then followed by a sharpening process to enhance details. The Gaussian blur was implemented using a symmetric kernel. This blurring helps in reducing image noise and minor details. In the sharpening phase, a predefined 3x3 kernel accentuates the edges by amplifying the high-frequency components, effectively countering the prior blurring and restoring clarity to the image.

The results are presented below. While this approach is less computationally intensive, its didn't produce any significant improvement over base GS output. Attempting to use stronger kernels eventually led to fragmented output. In addition, its PSNR on the test set  was similar to that of base GS.

<div style="text-align: center">
    <img src="figures/cv2_output.png" />
</div>

### (2) DeblurGAN
In addition, we've tried using the [DeblurGAN](https://arxiv.org/abs/1711.07064v4) model in order to unblur images. However, the model produced images where the object was translated/transformed from the input image. While minor, this output led to optimization of Gaussians such that two objects were rendered, while obviously only a single object is truly present. While the diffusion model isn't free of this symptom, it's less acute. 


# Conclusion
Denoising-models can provide supervision for Gaussian Splatting models in areas far from input training, thus allowing relative long-range extrapolation with respect to training. However, the final GS-model is only as strong as the denoising model. As we've seen in our works, artifcats introduced by the denoising models were evidently present in the output of the final GS-model. Therefore, stronger/better-trained diffusion models can be replaced easily into this pipeline, and generate better results. In addition, regularizations and constraints can be imposed on the output of the diffusion model, further increasing the consitency between data from input images, and generated data from far views.